# Программирование на языке Python
## Семинар 25. Функции, замыкание и декораторы
Полезная информация для понимания темы:

1. [Функции - это объекты первого класса](https://habr.com/ru/companies/otus/articles/725374/);
2. [Декораторы](https://habr.com/ru/companies/otus/articles/727590/);
3. [Некоторые встроенные декораторы](https://www.geeksforgeeks.org/top-python-built-in-decorators-that-optimize-python-code-significantly/).

#### План занятия
1. Немного теории;
2. Некоторые полезные декораторы;
3. Практикум: настроим логи с помощью декоратора.

#### Лексическое замыкание
Это способность функции "запоминать" состояние родительской функции.

Рассмотрим на примере:

In [1]:
def enclosing_function():  # функция высшего порядка
    def show_local_state():
        return 'local state'

    return show_local_state

In [3]:
fun_from_fun = enclosing_function()  # функция сама по себе может быть возвращена другой функцией
fun_from_fun()

'local state'

In [5]:
enclosing_function

<function __main__.enclosing_function()>

In [4]:
fun_from_fun

<function __main__.enclosing_function.<locals>.show_local_state()>

In [6]:
callable(fun_from_fun)

True

In [7]:
hasattr(fun_from_fun, '__call__')

True

In [8]:
class SomeCallable:
    def __init__(self):
        pass

    def __call__(self):
        return 'I am callable object'

In [9]:
sc = SomeCallable()

In [10]:
sc()

'I am callable object'

**Задание 1**

Поменяйте предложенный ниже код, перенеся условную конструкцию в функцию.

In [15]:
const = 48

def int1_to_str(number: int) -> str:  # baseline function
    return chr(number + const)

def int_to_str(number: int) -> str:  # while
    a = abs(number)

    if number != 0:
        result = ''
        
        while a != 0:  # a > 0
            num = a % 10
            result += int1_to_str(num)  # result = result + int1_to_str(num)
            a //= 10  # a = a // 10
        
        result = result[::-1]  # result = '-' * (some_int < 0) + result[::-1]
        
        if number < 0:
            result = '-' + result

    else:
        result = int1_to_str(number)  # result = '0'
    
    return result

def int_to_str_r(number: int) -> str:  # recursion
    if number < 10:
        return int1_to_str(number)
    else:
        return int_to_str_r(number // 10) + int1_to_str(number % 10)


# select appropriate function depending on number size
number = 892834282939493932

if number // 1e6 > 0:
    result = int_to_str(number)
else:
    result = int_to_str_r(number)

print(result)

892834282939493932


In [16]:
def get_converter(number):
    def int1_to_str(number: int) -> str:  # baseline function
        return chr(number + const)
    
    def int_to_str(number: int) -> str:  # while
        a = abs(number)
    
        if number != 0:
            result = ''
            
            while a != 0:  # a > 0
                num = a % 10
                result += int1_to_str(num)  # result = result + int1_to_str(num)
                a //= 10  # a = a // 10
            
            result = result[::-1]  # result = '-' * (some_int < 0) + result[::-1]
            
            if number < 0:
                result = '-' + result
    
        else:
            result = int1_to_str(number)  # result = '0'
        
        return result
    
    def int_to_str_r(number: int) -> str:  # recursion
        if number < 10:
            return int1_to_str(number)
        else:
            return int_to_str_r(number // 10) + int1_to_str(number % 10)
        
    if number // 1e6 > 0:
        return int_to_str
    else:
        return int_to_str_r

In [18]:
converter = get_converter(number=number)
converter(number)

'892834282939493932'

In [19]:
converter

<function __main__.get_converter.<locals>.int_to_str(number: int) -> str>

Теперь рассмотрим непосредственно замыкание:

In [20]:
def enclosing_function(mode):  # функция высшего порядка
    if mode == 1:
        local_state = 'local state 1'
    elif mode == 2:
        local_state = 'local state 2'
        
    def show_local_state():  # функция "вытаскивает в себя" локальное состояние функции enclosing_function через return
        return local_state

    return show_local_state

In [21]:
local_func = enclosing_function(1)
local_func()

'local state 1'

In [22]:
local_func

<function __main__.enclosing_function.<locals>.show_local_state()>

In [24]:
local_func = enclosing_function(2)
local_func()

'local state 2'

In [25]:
local_func

<function __main__.enclosing_function.<locals>.show_local_state()>

**Задание 2**

Реализуйте ООП-решение задачи предыдущего семинара в парадигме функционального программирования. Обязательно используйте замыкание.

In [34]:
import numpy as np
import pandas as pd

def get_transformer(ttype='norm'):
    _valid_structures = [list, tuple, np.ndarray, pd.DataFrame]
    _valid_dtypes = [int, float]

    if ttype=='norm':
        from norm_functions import get_params, apply_params
    else:
        from std_functions import get_params, apply_params

    def _validate_data(data):
        # check data structure
        structure = type(data)
        if structure not in _valid_structures:
            raise ValueError(f'invalid structure {structure}, use only authorized structures: {_valid_structures}!')
        
        # <data> -> np.ndarray
        if structure in [list, tuple]:
            data = np.array(data)
        elif structure == pd.DataFrame:
            data = data.values
        
        # check dtype
        if data.dtype not in _valid_dtypes:
            raise ValueError(f'invalid dtype {data.dtype}, use only authorized dtypes: {_valid_dtypes}!')

        # shape
        if data.ndim != 2:
            raise ValueError(f'invalid shape {data.ndim}, must be 2!')

        return data

    def fit(data):
        # validate data
        data_valid = _validate_data(data)
        
        # compute params
        params = get_params(data_valid)
            
        # save params
        return params

    def transform(data, params) -> pd.DataFrame: 
        # validate data
        data_valid = _validate_data(data)

        # apply params to data
        data_transformed = apply_params(data_valid, params)

        return data_transformed

    return fit, transform

In [35]:
X = np.random.uniform(-10, 10, (100, 2))
y = np.random.normal(0, 1, 100)

X_new = np.random.uniform(-10, 10, (50, 2))

fit, transform = get_transformer(ttype='norm')

params = fit(X)
X_transformed = transform(X, params)

In [31]:
fit, transform

(<function __main__.get_transformer.<locals>.fit(data)>,
 <function __main__.get_transformer.<locals>.transform(data, params) -> pandas.core.frame.DataFrame>)

#### Декораторы
Декоратор - это, в сущности, функция, которая принимает на вход другую функцию и возвращает ее модификацию:

In [37]:
from datetime import datetime

def what_time_is_it():
    now = datetime.now()

    return now

In [38]:
what_time_is_it

<function __main__.what_time_is_it()>

In [39]:
what_time_is_it()

datetime.datetime(2023, 11, 29, 18, 57, 28, 100784)

In [40]:
def prettifier(func):
    def wrapper():
        now = func()
        pretty_now = now.strftime('%d.%m.%Y %H:%M:%S')
        
        return pretty_now
        
    return wrapper

Без синтаксического сахара:

In [41]:
what_time_is_it = prettifier(what_time_is_it)

In [42]:
what_time_is_it

<function __main__.prettifier.<locals>.wrapper()>

С синтаксическим сахаром:

In [44]:
from datetime import datetime

@prettifier
def what_time_is_it():
    now = datetime.now()

    return now

In [45]:
what_time_is_it

<function __main__.prettifier.<locals>.wrapper()>

In [46]:
what_time_is_it()

'29.11.2023 19:02:57'

А как с аргументами?

In [47]:
from pathlib import Path
import openpyxl

def parse_info(path: Path) -> dict:
    wb = openpyxl.load_workbook(path)
    ws = wb.active

    info_dict = {
        (row[0].value, row[1].value): {'fio': row[2].value, 'email': row[3].value}
        for row
        in ws.iter_rows()
        if row[0].value != 'Блок'
    }

    wb.close()

    return info_dict

In [48]:
info_path = Path('../Занятие 5/sources/Справочник.xlsx')

parse_info(info_path)

{('Блок 1', 'Департамент A'): {'fio': 'Довгополый Иоанн Алексеевич',
  'email': 'ioannundseineigentum@gmail.com'},
 ('Блок 1', 'Департамент B'): {'fio': 'Довгополый Иоанн Алексеевич',
  'email': 'ioannundseineigentum@gmail.com'},
 ('Блок 1', 'Департамент C'): {'fio': 'Довгополый Иоанн Алексеевич',
  'email': 'ioannundseineigentum@gmail.com'},
 ('Блок 2', 'Департамент A'): {'fio': 'Довгополый Иоанн Алексеевич',
  'email': 'ioannundseineigentum@gmail.com'},
 ('Блок 2', 'Департамент B'): {'fio': 'Довгополый Иоанн Алексеевич',
  'email': 'ioannundseineigentum@gmail.com'},
 ('Блок 2', 'Департамент C'): {'fio': 'Довгополый Иоанн Алексеевич',
  'email': 'ioannundseineigentum@gmail.com'},
 ('Блок 2', 'Департамент D'): {'fio': 'Довгополый Иоанн Алексеевич',
  'email': 'ioannundseineigentum@gmail.com'},
 ('Блок 2', 'Департамент E'): {'fio': 'Довгополый Иоанн Алексеевич',
  'email': 'ioannundseineigentum@gmail.com'},
 ('Блок 2', 'Департамент F'): {'fio': 'Довгополый Иоанн Алексеевич',
  'email': 

In [56]:
import pandas as pd

def pandify(func):
    def wrapper(*args, **kwargs):
        dictlike = func(*args, **kwargs)
        
        list_to_pandas = [{'block': key[0], 'department': key[1], **value} for key, value in dictlike.items()]

        df = pd.DataFrame(list_to_pandas)

        return df

    return wrapper

In [49]:
def some_func(*args, **kwargs):
    return args, kwargs

In [51]:
some_func(1, 2, 3, a='a', b='b')

((1, 2, 3), {'a': 'a', 'b': 'b'})

In [52]:
def some_func(a, b):
    return a + b

In [53]:
values = {'a': 6, 'b': 11}

In [55]:
some_func(**values)

17

In [57]:
@pandify
def parse_info(path: Path) -> dict:
    wb = openpyxl.load_workbook(path)
    ws = wb.active

    info_dict = {
        (row[0].value, row[1].value): {'fio': row[2].value, 'email': row[3].value}
        for row
        in ws.iter_rows()
        if row[0].value != 'Блок'
    }

    wb.close()

    return info_dict

In [58]:
info_path = Path('../Занятие 5/sources/Справочник.xlsx')

parse_info(info_path)

,block,department,fio,email
0,Блок 1,Департамент A,Довгополый Иоанн Алексеевич,ioannundseineigentum@gmail.com
1,Блок 1,Департамент B,Довгополый Иоанн Алексеевич,ioannundseineigentum@gmail.com
2,Блок 1,Департамент C,Довгополый Иоанн Алексеевич,ioannundseineigentum@gmail.com
3,Блок 2,Департамент A,Довгополый Иоанн Алексеевич,ioannundseineigentum@gmail.com
4,Блок 2,Департамент B,Довгополый Иоанн Алексеевич,ioannundseineigentum@gmail.com
5,Блок 2,Департамент C,Довгополый Иоанн Алексеевич,ioannundseineigentum@gmail.com
6,Блок 2,Департамент D,Довгополый Иоанн Алексеевич,ioannundseineigentum@gmail.com
7,Блок 2,Департамент E,Довгополый Иоанн Алексеевич,ioannundseineigentum@gmail.com
8,Блок 2,Департамент F,Довгополый Иоанн Алексеевич,ioannundseineigentum@gmail.com
9,Блок 2,Департамент G,Довгополый Иоанн Алексеевич,ioannundseineigentum@gmail.com


**Как быть с docstring?**

In [60]:
def func(n):
    """
    This function powers input n by 2
    """
    return n ** 2

In [61]:
func.__doc__

'\n    This function powers input n by 2\n    '

In [63]:
def divider(func):
    def wrapper(*args, **kwargs):
        result = func(*args, **kwargs)
        return result ** (1 / 2)
    return wrapper

In [64]:
@divider
def func(n):
    return n ** 2

In [65]:
?func

Signature: func(*args, **kwargs)
Docstring: <no docstring>
File:      /var/folders/h6/lv17v1r10lz21g745pd6774h0tgc4z/T/ipykernel_24880/3284642871.py
Type:      function

**Решение**

In [78]:
def func(n):
    """
    This function powers input n by 2
    """
    return n ** 2

In [79]:
import functools

def divider(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        result = func(*args, **kwargs)
        return result ** (1 / 2)
    return wrapper

In [82]:
@divider
def func(n):
    """
    This function powers input n by 2
    """
    return n ** 2

In [83]:
?func

Signature: func(n)
Docstring: This function powers input n by 2
File:      /var/folders/h6/lv17v1r10lz21g745pd6774h0tgc4z/T/ipykernel_24880/2309582757.py
Type:      function

In [72]:
def func(n):
    """
    This function powers input n by 2
    """
    return n ** 2

In [88]:
def divider(func):
    doc = func.__doc__
    
    def wrapper(*args, **kwargs):
        result = func(*args, **kwargs)
        return result ** (1 / 2)

    wrapper.doc = doc
    
    return wrapper

In [89]:
@divider
def func(n):
    """
    This function powers input n by 2
    """
    return n ** 2

In [90]:
?func

Signature: func(*args, **kwargs)
Docstring: <no docstring>
File:      /var/folders/h6/lv17v1r10lz21g745pd6774h0tgc4z/T/ipykernel_24880/126067108.py
Type:      function

#### Некоторые полезные декораторы

1. @classmethod (метод для класса, подразумевает аргумент cls);
2. @staticmethod (метод для класса, не привязан классу никак кроме namespace);
3. @abstractmethod (только обозначить наличие метода);
4. @typing.final (обозначить, что от класса нельзя наследоваться);
5. @lru_cache.

In [91]:
from abc import ABC, abstractmethod

In [94]:
class ParentOnly(ABC):
    @abstractmethod
    def _get_params(self, data):  # must be specified in child class
        pass

    @abstractmethod
    def _apply_params(self, data):  # must be specified in child class
        pass

In [95]:
po = ParentOnly()

TypeError: Can't instantiate abstract class ParentOnly with abstract methods _apply_params, _get_params

In [104]:
import typing

@typing.final
class FinalDescendant:
    def __init__(self):
        super().__init__()

In [103]:
?FinalDescendant

Init signature: FinalDescendant()
Docstring:     
Helper class that provides a standard way to create an ABC using
inheritance.
Type:           ABCMeta
Subclasses:     

In [105]:
class FinalFinalDescendant(FinalDescendant):
    pass

In [106]:
f = FinalFinalDescendant()

#### Практика
Напишите декоратор для сохранения логов в файл stdout.txt. Модифицируйте проект по рассылке отчетов.

In [107]:
import sys

In [109]:
def printer(number):
    print(number)

def printer2(number):
    print(str(number) * 10)

In [110]:
number = 9

printer(number)
printer2(number)

9
9999999999


In [1]:
import sys

def catcher(func):
    def wrapper(*args, **kwargs):
        original_stdout = sys.stdout

        with open('stdout.txt', 'a') as file:
            sys.stdout = file
            func(*args, **kwargs)

        sys.stdout = original_stdout

    return wrapper

In [2]:
@catcher
def printer(number):
    print(number)

@catcher
def printer2(number):
    print(str(number) * 10)

In [3]:
for i in range(100):
    printer(i)
    printer2(i)

In [108]:
sys.stdout

In [ ]:
# наш код здесь